## Google Maps Geocoding API
Creating API keys:
The API key is a unique identifier that authenticates requests associated with your project for usage and billing purposes. You must have at least one API key associated with your project.

To create an API key:
https://developers.google.com/maps/documentation/geocoding/get-api-key.

# Imports

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

# Initialize the Google Maps Geocoding API endpoint and API key

In [ ]:
API_ENDPOINT = 'https://maps.googleapis.com/maps/api/geocode/json'
API_KEY= input('Please enter your Google Maps API Key and press Enter:')
if API_KEY != '':
    print(f'API Key Entered successfully, Last seven charaters: {API_KEY[len(API_KEY)-7:]}')
else:
    print('Enter valid API Key')
    exit()

# Import the CSV files

In [ ]:
import glob
import os
import pandas as pd

all_files = glob.glob('./data/output/chatgpt_results_*.csv')
df = pd.concat((pd.read_csv(f) for f in all_files))

In [ ]:
df.head(5)

# Create new columns for latitude, longitude, and full address

In [ ]:
# Create new columns for latitude, longitude, and full address
df['Google_Maps_Latitude'] = ''
df['Google_Maps_Longitude'] = ''
df['Google_Maps_Full Address'] = ''
df.head(1)

In [ ]:
# Geocode each address using the Google Maps Geocoding API
for i in tqdm(range(len(df))):
    try:
        full_address = f"{df['Institute_chatGPT'][i]}, {df['Country_chatGPT'][i]}"
        params = {'address': full_address, 'key': API_KEY}
        response = requests.get(API_ENDPOINT, params=params)
        json_response = response.json()
        if json_response['status'] == 'OK':
            df.loc[i, 'Google_Maps_Latitude'] = json_response['results'][0]['geometry']['location']['lat']
            df.loc[i, 'Google_Maps_Longitude'] = json_response['results'][0]['geometry']['location']['lng']
            df.loc[i, 'Google_Maps_Full Address'] = json_response['results'][0]['formatted_address']
    except:
        # If geocoding fails, set the latitude, longitude, and full address columns to empty strings
        df.loc[i, 'Google_Maps_Latitude'] = ''
        df.loc[i, 'Google_Maps_Longitude'] = ''
        df.loc[i, 'Google_Maps_Full Address'] = ''

In [ ]:
df.head(5)

# Export

In [ ]:
df.to_csv('./data/output/sample_data_chatgpt_compare_googlemap_Address.csv', index=False)